



#**Aivle 스쿨 지원 질문, 답변 챗봇 만들기**
# 단계3 : 모델링

## 0.미션

* 다음 두가지 챗봇을 만들고 비교해 봅시다.
* 챗봇. Word2Vec 임베딩 벡터 기반 머신러닝 분류 모델링
    * Word2Vec 모델을 만들고 임베딩 벡터를 생성합니다.
    * 임베딩 벡터를 이용하여 intent를 분류하는 모델링을 수행합니다.
        * 이때, LightGBM을 추천하지만, 다른 알고리즘을 이용할수 있습니다.
    * 예측된 intent의 답변 중 임의의 하나를 선정하여 출력합니다.

## 1.환경준비

### (1)라이브러리 설치

#### 1) 자연어 처리를 위한 라이브러리

In [1]:
#gensim은 자연어 처리를 위한 오픈소스 라이브러리입니다. 토픽 모델링, 워드 임베딩 등 다양한 자연어 처리 기능을 제공
!pip install gensim

#### 2) 형태소 분석을 위한 라이브러리
* 참조 : https://konlpy.org/en/latest/install/

In [2]:
# mecab 설치를 위한 관련 패키지 설치
!apt-get install curl git
!apt-get install build-essential
!apt-get install cmake
!apt-get install g++
!apt-get install flex
!apt-get install bison
!apt-get install python-dev
!pip install cython
!pip install mecab-python

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
curl is already the newest version (7.81.0-1ubuntu1.16).
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
g++ is already the newest version (4:11.2.0-1ubuntu1).
g++ set to manually installed.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
Reading package lists... Done
B

In [3]:
# 형태소 기반 토크나이징 (Konlpy)
!python3 -m pip install konlpy
# mecab (ubuntu: linux, mac os 기준)
# 다른 os 설치 방법 및 자세한 내용은 다음 참고: https://konlpy.org/ko/latest/install/#id1
!bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
# !pip install mecab

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 43.5 MB/s eta 0:00:00
Install mecab-ko
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 1381k  100 1381k    0     0   982k      0  0:00:01  0:00:01 --:--:-- 1699k
mecab-0.996-ko-0.9.2/
mecab-0.996-ko-0.9.2/example/
mecab-0.996-ko-0.9.2/example/example.cpp
mecab-0.996-ko-0.9.2/example/example_lattice.cpp
mecab-0.996-ko-0.9.2/example/example_lattice.c
mecab-0.996-ko-0.9.2/example/example.c
mecab-0.996-ko-0.9.2/example/thread_test.cpp
mecab-0.996-ko-0.9.2/mecab-config.in
mecab-0.996-ko-0.9.2/man/
mecab-0.996-ko-0.9.2/man/Makefile.am
mecab-0.996-ko-0.9.2/man/mecab.1
mecab-0.996-ko-0.9.2/man/Makefile.in
mecab-0.996-ko-0.9.2/mecab.iss.in
mecab-0.996-ko-0.9.2/confi

In [4]:
from konlpy.tag import Okt, Komoran, Mecab, Hannanum, Kkma

# 다양한 토크나이저를 사용할 수 있는 함수
def get_tokenizer(tokenizer_name):
    if tokenizer_name == "komoran":
        tokenizer = Komoran()
    elif tokenizer_name == "okt":
        tokenizer = Okt()
    elif tokenizer_name == "mecab":
        tokenizer = Mecab()
    elif tokenizer_name == "hannanum":
        tokenizer = Hannanum()
    else:
        tokenizer = Kkma()

    return tokenizer

In [5]:
def tokenize(tokenizer_name, original_sent, nouns=False):
    tokenizer = get_tokenizer(tokenizer_name)

    cleaned_sent = re.sub(r'[^\sa-zA-Z가-힣]', '', original_sent).lower().strip()
    cleaned_sent = re.sub(r'\s+', ' ', cleaned_sent)

    tokens = tokenizer.morphs(cleaned_sent, norm=True, stem=True)

    tokens = [('kt' if word in ['ktt', 'kttt'] else word) for word in tokens]

    stop_words = ['mmm', 'q']
    tokens = [token for token in tokens if token not in stop_words]

    return tokens

### (2) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [6]:
import numpy as np

import joblib

import re

from lightgbm import LGBMClassifier
from sklearn.metrics import *

### (3) 데이터 로딩
* 전처리 단계에서 생성한 데이터들을 로딩합니다.
    * eda_data, test
* Google Colab 환경에서 진행을 권장합니다.
    * 구글 드라이브 바로 밑에 project 폴더를 만들고,
    * 데이터 파일을 복사해 넣습니다.

#### 1) Google Colab 환경 구축

* 구글 드라이브 연결

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
%cd /content/drive/MyDrive/KT_AIVLE/2023.10.30_미니프로젝트6차

/content/drive/MyDrive/KT_AIVLE/2023.10.30_미니프로젝트6차


#### 2) 저장된 데이터 읽어오기
* 저장된 .pkl 파일들을 불러옵니다.

In [9]:
import joblib
train = joblib.load('eda_data')
test = joblib.load('test')

## 2.챗봇1

* **상세요구사항**
    * Word2Vec을 활용한 LightGBM 모델링(intent 분류)
        * Word2Vec을 이용하여 임베딩벡터 생성하기
            * Word Embedding으로 문장벡터 구하기
        * 임베딩 벡터를 이용하여 ML기반 모델링 수행하기
            * LightGBM 권장(다른 알고리즘을 이용할수 있습니다.)
    * 챗봇 : 모델의 예측결과(intent)에 따라 답변하는 챗봇 만들기
        * 질문을 입력받아, 답변하는 함수 생성

### (1) Word2Vec을 이용하여 임베딩벡터 생성하기
* 'okt' 형태소 분석기를 이용하여 문장을 tokenize
    * Word2Vec 모델을 만들기 위해서 입력 데이터는 리스트 형태여야 합니다.
    * 그래서 다시 리스트로 저장되도록 토크나이즈 해 봅시다.
* Word Embedding으로 문장벡터를 생성합니다.
    * 먼저 Word2Vec 모델을 만들고, train의 질문들을 문장벡터로 만듭시다.


#### 1) 'Okt' 형태소 분석기를 이용하여 문장을 tokenize

In [10]:
train['token'] = train['Q'].apply(lambda x : tokenize('okt', x))
test['token'] = test['check'].apply(lambda x : tokenize('okt', x))
train['token']

0                 [아르바이트, 를, 하다, 다니다, 때, 는, 수강, 하다, 수, 없다]
1                       [아르바이트, 를, 하고도, 교육, 을, 시키다, 수, 없다]
2        [아르바이트, 를, 많이, 하고만, 있다, 분, 의, 경우, 교육, 을, 수강, 하...
3        [아르바이트, 를, 하다, 있다, 분, 의, 경우, 교육, 을, 제대로, 제대로, ...
4        [아르바이트, 를, 하고만, 남아, 있다, 분, 의, 경우, 교육, 을, 수강, 하...
                               ...                        
15715                                                [생리통]
15716                                            [만성, 생리통]
15717                               [프리랜서, 는, 미취, 업자, 인가요]
15718                          [그렇다, 프리랜서, 는, 미취, 업자, 인가요]
15719                               [프리랜서, 는, 미취, 업자, 인가요]
Name: token, Length: 15720, dtype: object

In [11]:
train.drop_duplicates(subset=['token'], inplace=True)
train.reset_index(drop=True, inplace=True)
train

,intent,Q,A,type,token
0,32,아르바이트를 하고 다닐 때는 수강할 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1,"[아르바이트, 를, 하다, 다니다, 때, 는, 수강, 하다, 수, 없다]"
1,32,아르바이트를 하고도 교육을 시킬 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1,"[아르바이트, 를, 하고도, 교육, 을, 시키다, 수, 없다]"
2,32,아르바이트를 많이 하고만 있는 분의 경우 교육을 수강할 할 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1,"[아르바이트, 를, 많이, 하고만, 있다, 분, 의, 경우, 교육, 을, 수강, 하..."
3,32,아르바이트를 하고 있는 분의 경우 교육을 제대로 제대로 받아 수강할 할 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1,"[아르바이트, 를, 하다, 있다, 분, 의, 경우, 교육, 을, 제대로, 제대로, ..."
4,32,아르바이트를 하고만 남아 있는 분의 경우 교육을 수강할 수 없나요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1,"[아르바이트, 를, 하고만, 남아, 있다, 분, 의, 경우, 교육, 을, 수강, 하..."
...,...,...,...,...,...
15125,47,교육기간과 기간 동안의 테스트 및 시험에서는 정해진 시간대 안에 문제를 풀어야만 한...,"KT 에이블스쿨은 자기주도적인 학습을 장려하고 있으며, 교과목 종료 시 셀프테스트를...",1,"[교육, 기간, 과, 기간, 동안, 의, 테스트, 및, 시험, 에서는, 정해진, 시..."
15126,8,생리통,힘들겠어요.,0,[생리통]
15127,8,만성 생리통,힘들겠어요.,0,"[만성, 생리통]"
15128,32,프리랜서는? 미취업자인가요?,"KT 에이블스쿨은 미취업자를 대상으로 하며, 교육 시작일 기준 재직자는 지원이 불가...",1,"[프리랜서, 는, 미취, 업자, 인가요]"


In [12]:
train_sent = train['token'].tolist()
test_sent = test['token'].tolist()

#### 2) Word Embedding으로 문장벡터 구하기
* Word2Vec
    * 위에서 저장한 입력 데이터를 사용하여 Word2Vec 모델이 생성
    * 모델은 size(단어 벡터의 차원),
    * window(컨텍스트 창의 크기),
    * max_vocab_size(고려할 최대 어휘 크기),
    * min_count(포함할 단어의 최소 빈도)와 같은 특정 하이퍼파라미터로 훈련됩니다.
    * sg : 사용할 훈련 알고리즘 - 1은 skip-gram, 0은 CBOW )

In [13]:
from gensim.models import Word2Vec

# Word2Vec 모델 생성
wv_model = Word2Vec(sentences=train_sent, vector_size=300, window=5, min_count=3, sg=1)

* Word2Vec 모델로부터 데이터를 벡터화하기 위한 함수 생성

In [14]:
# Word2Vec 모델로부터 하나의 문장을 벡터화 시키는 함수 생성
def get_sent_embedding(model, embedding_size, tokenized_words):
    # 임베딩 벡터를 0으로 초기화
    feature_vec = np.zeros((embedding_size,), dtype='float32')
    # 단어 개수 초기화
    n_words = 0
    # 모델 단어 집합 생성
    index2word_set = set(model.wv.key_to_index.keys())
    # 문장의 단어들을 하나씩 반복
    for word in tokenized_words:
        # 모델 단어 집합에 해당하는 단어일 경우에만
        if word in index2word_set:
            # 단어 개수 1 증가
            n_words += 1
            # 임베딩 벡터에 해당 단어의 벡터를 더함
            feature_vec = np.add(feature_vec, model.wv.get_vector(word))
    # 단어 개수가 0보다 큰 경우 벡터를 단어 개수로 나눠줌 (평균 임베딩 벡터 계산)
    if (n_words > 0):
        feature_vec = np.divide(feature_vec, n_words)
    return feature_vec

In [15]:
# 문장벡터 데이터 셋 만들기
def get_dataset(sentences, model, num_features):
    dataset = list()

    # 각 문장을 벡터화해서 리스트에 저장
    for sent in sentences:
        dataset.append(get_sent_embedding(model, num_features, sent))

    # 리스트를 numpy 배열로 변환하여 반환
    sent_embedding_vectors = np.stack(dataset)

    return sent_embedding_vectors

* 이제 학습데이터의 Q를 Word2Vec 모델을 사용하여 벡터화 합니다.

In [16]:
# 학습 데이터의 문장들을 Word2Vec 모델을 사용하여 벡터화
train_data_vecs = get_dataset(train_sent, wv_model, 300)

In [17]:
train_data_vecs

array([[-0.12932925,  0.12496203, -0.04419575, ..., -0.14466766,
         0.02415203, -0.2652971 ],
       [-0.10934641,  0.05285329, -0.07594515, ..., -0.18179598,
        -0.03374147, -0.1489808 ],
       [-0.12697051,  0.06259813, -0.07899575, ..., -0.13807453,
         0.04915848, -0.24085756],
       ...,
       [ 0.00292194,  0.03511798, -0.08686744, ..., -0.06693956,
         0.02977685, -0.02243573],
       [-0.15448518,  0.2265354 , -0.08239533, ..., -0.10471097,
         0.1413401 , -0.22576757],
       [-0.16851117,  0.24027407, -0.07437655, ..., -0.09848476,
         0.1158777 , -0.20485471]], dtype=float32)

* 훈련된 Word2Vec 모델을 사용하여 문장 목록에 대한 문장 임베딩을 생성하고 이를 2차원 numpy 배열에 저장합니다.
* 그런 다음 이러한 임베딩을 다양한 기계 학습 모델의 입력 기능으로 사용할 수 있습니다

### (2) 분류 모델링
* 데이터 분할
    * x, y
        * x : 이전 단계에서 저장된 임베딩벡터
        * y : intent 값들
    * train, val
        * train_test_split 활용
* 머신러닝 모델링
    * lightGBM, RandomForest 등을 활용하여 학습
    * 필요하다면 hyper parameter 튜닝을 시도해도 좋습니다.
* validation set으로 검증해 봅시다.

In [18]:
from sklearn.model_selection import train_test_split

# X와 y 데이터 분리
x=train_data_vecs
y=train['intent']

# Train-Test split
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, shuffle=True, stratify=train['intent'], random_state=42)

In [19]:
x_train.shape, x_val.shape, y_train.shape, y_val.shape

((12104, 300), (3026, 300), (12104,), (3026,))

* 모델1

In [ ]:
# LightGBM 분류기 생성
model_lgbm=LGBMClassifier()
# 학습
model_lgbm.fit(x_train, y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.038652 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 76500
[LightGBM] [Info] Number of data points in the train set: 7044, number of used features: 300
[LightGBM] [Info] Start training from score -4.732797
[LightGBM] [Info] Start training from score -4.908688
[LightGBM] [Info] Start training from score -4.569472
[LightGBM] [Info] Start training from score -5.601835
[LightGBM] [Info] Start training from score -5.304583
[LightGBM] [Info] Start training from score -5.122262
[LightGBM] [Info] Start training from score -4.968111
[LightGBM] [Info] Start training from score -3.561614
[LightGBM] [Info] Start training from score -5.724437
[LightGBM] [Info] Start training from score -5.425944
[LightGBM] [Info] Start training from score -5.394196
[LightGBM] [Info] Start training from score -5.492636
[LightGBM] [Info] Start training from score -4.765587
[LightGB

LGBMClassifier()

In [ ]:
# 예측 및 검증
y_pred_lgbm=model_lgbm.predict(x_val)
print(confusion_matrix(y_val, y_pred_lgbm))
print(classification_report(y_val, y_pred_lgbm))

[[ 4  0  1 ...  0  0  0]
 [ 0  1  0 ...  0  0  0]
 [ 1  0  3 ...  0  0  0]
 ...
 [ 0  0  0 ... 16  2  0]
 [ 0  0  0 ...  0 31  0]
 [ 0  0  4 ...  0  0  2]]
              precision    recall  f1-score   support

           1       0.14      0.27      0.19        15
           2       0.14      0.08      0.10        13
           3       0.12      0.17      0.14        18
           4       0.00      0.00      0.00         6
           5       1.00      0.11      0.20         9
           6       1.00      0.55      0.71        11
           7       0.55      0.50      0.52        12
           8       0.17      0.16      0.16        50
           9       0.00      0.00      0.00         6
          10       0.00      0.00      0.00         8
          11       0.57      0.50      0.53         8
          12       0.00      0.00      0.00         7
          13       0.00      0.00      0.00        15
          14       0.00      0.00      0.00         4
          15       0.29      0.20

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


* 모델2

In [ ]:
# RandomForest
from sklearn.ensemble import RandomForestClassifier
model_rf=RandomForestClassifier()
# 학습
model_rf.fit(x_train, y_train)

RandomForestClassifier()

In [ ]:
# 예측 및 검증
y_pred_rf=model_rf.predict(x_val)
print(confusion_matrix(y_val, y_pred_rf))
print(classification_report(y_val, y_pred_rf))

[[25  0  0 ...  0  0  0]
 [ 0 24  0 ...  0  0  0]
 [ 0  0 42 ...  0  0  0]
 ...
 [ 0  0  0 ... 56  0  0]
 [ 0  0  0 ...  0 76  0]
 [ 0  0  0 ...  0  0 74]]
              precision    recall  f1-score   support

           1       1.00      1.00      1.00        25
           2       0.92      0.92      0.92        26
           3       1.00      1.00      1.00        42
           4       0.93      0.88      0.90        16
           5       1.00      1.00      1.00        12
           6       1.00      0.93      0.96        14
           7       1.00      1.00      1.00        21
           8       0.86      0.97      0.91        94
           9       1.00      0.73      0.84        11
          10       1.00      1.00      1.00        12
          11       0.87      0.81      0.84        16
          12       1.00      1.00      1.00        20
          13       1.00      0.88      0.93        24
          14       0.80      0.73      0.76        11
          15       0.89      1.00

* 모델 저장하기

In [ ]:
joblib.dump(model_lgbm, 'model_lgbm.pkl')
joblib.dump(model_rf, 'model_rf.pkl')

['model_rf.pkl']

### (3) 챗봇 구축

# 챗봇1
* **상세요구사항**
    * 챗봇 flow : input 질문 -> 분류 모델로 intent 예측 --> intent에 해당하는 답변 출력
        * 하나의 intent 에는 여러 답변이 있습니다. 이중 한가지를 랜덤하게 선택합니다.

#### 1) 데이터 중 하나에 대해서 테스트

In [24]:
%cd py-hanspell

/content/drive/MyDrive/KT_AIVLE/2023.10.30_미니프로젝트6차/py-hanspell


In [25]:
!python setup.py install

/usr/local/lib/python3.10/dist-packages/setuptools/dist.py:755: SetuptoolsDeprecationWarning: Invalid dash-separated options
!!

        ********************************************************************************
        Usage of dash-separated 'description-file' will not be supported in future
        versions. Please use the underscore name 'description_file' instead.

        This deprecation is overdue, please update your project and remove deprecated
        calls to avoid build errors in the future.

        See https://setuptools.pypa.io/en/latest/userguide/declarative_config.html for details.
        ********************************************************************************

!!
  opt = self.warn_dash_deprecation(opt, section)
running install
/usr/local/lib/python3.10/dist-packages/setuptools/_distutils/cmd.py:66: SetuptoolsDeprecationWarning: setup.py install is deprecated.
!!

        ********************************************************************************
 

In [26]:
from hanspell import spell_checker

def preprocess(original_sent):
    check_sent = spell_checker.check(original_sent)
    token_sent = tokenize('okt', check_sent.checked)
    return token_sent

In [ ]:
input = preprocess('넘무아파요힘드러요')
input

['너무', '아프다', '힘들다']

In [ ]:
# 입력문장 벡터화
input_vecs = get_dataset([input], wv_model, 300)
input_vecs

array([[-4.68256138e-02,  8.54173079e-02, -9.96215921e-03,
         1.51372686e-01,  1.27799615e-01, -3.14907506e-02,
        -1.88526511e-01,  3.90131950e-01,  1.03435814e-01,
        -1.33390836e-02, -3.76310907e-02, -6.90232590e-02,
         6.55901507e-02,  1.06904805e-01,  1.83797274e-02,
        -2.74372045e-02, -4.52277549e-02, -6.31524324e-02,
         4.81941588e-02, -1.33486092e-01,  5.01053175e-03,
        -4.53771092e-02,  1.09706670e-01, -7.20894784e-02,
         2.71544810e-02,  1.46205723e-01, -4.07728553e-02,
        -6.69053802e-03,  1.42948553e-01, -2.58277450e-02,
        -2.20963255e-01, -2.25391984e-01,  1.03776611e-01,
         1.88204005e-01,  5.79203665e-02,  3.60073633e-02,
        -2.40681618e-02, -1.23140588e-02,  6.49922490e-02,
        -2.29116619e-01, -3.65725368e-01, -3.01163346e-02,
        -6.14882894e-02, -1.48113459e-01, -1.75655112e-01,
         1.13651164e-01,  3.31718214e-02,  1.01495171e-02,
        -2.05190971e-01, -3.07217496e-03,  2.22633004e-0

In [ ]:
# lgbm 모델을 이용하여 intent 예측
pred=model_lgbm.predict(input_vecs)
pred[0]

38

In [ ]:
train.loc[train['intent']==pred[0]]['A'].unique()

array(['현재 KT 에이블스쿨은 KT 채용 인적성검사와 같은 과목의 인적성검사를 운영하고 있습니다.'], dtype=object)

In [ ]:
# rf 모델을 이용하여 intent 예측
pred2=model_rf.predict(input_vecs)
pred2[0]

23

In [ ]:
train.loc[train['intent']==pred2[0]]['A'].unique()

array(['술은 적당히 즐기세요.', '술 너무 많이 드시지 마세요.', '술 안 마셔도 놀 수 있어요.', '술이 웬수예요.'],
      dtype=object)

In [ ]:
# 배열 내 랜덤 선택
np.random.choice(train.loc[train['intent']==pred2[0]]['A'].unique())

'술 안 마셔도 놀 수 있어요.'

#### 2) 챗봇 함수 만들기
* 테스트 코드를 바탕으로 질문을 받아 답변을 하는 함수를 생성합시다.
* 성능이 좋은 모델 사용.

In [ ]:
def get_answer(question, model_rf, train):
    # 전처리 함수(preprocess)를 호출하고 나온 결과를 input 변수에 저장
    input = preprocess(question)

    # 입력 데이터를 word2vec 모델(wv_model)을 사용하여 벡터화
    input_vecs = get_dataset([input], wv_model, 300)

    # 랜덤포레스트 모델(model_rf)을 사용하여 예측 수행
    pred = model_rf.predict(input_vecs)

    # 예측 결과를 사용하여 대답 선택
    answer = np.random.choice(train.loc[train['intent']==pred[0]]['A'].unique())

    # 결과 출력
    print("질문 : ", question)
    print("답변 : ", answer, '\n\n==============================================\n')

In [ ]:
from ipywidgets import widgets
from IPython.display import display

# 질문을 입력받을 텍스트 입력 위젯 생성
question_text = widgets.Text(placeholder='질문을 입력하세요')
display(question_text)

# 텍스트 입력 위젯에 입력된 내용을 처리하는 함수 정의
def handle_submit(sender):
    user_question = question_text.value
    if user_question == '종료':
        print("종료합니다.")
        question_text.disabled = True  # 입력 위젯 비활성화
    else:
        get_answer(user_question, model_rf, train)
    question_text.value = ''  # 입력 위젯 초기화

# 텍스트 입력 위젯에 입력을 제출했을 때 처리할 함수 연결
question_text.on_submit(handle_submit)

Text(value='', placeholder='질문을 입력하세요')

질문 :  학교 수업과 교육을 병행할 수 있어?
답변 :  KT 에이블스쿨은 풀타임(09:00~18:00)으로 교육이 진행되며, 정해진 시간에 필수로 참여해야 합니다. 
교육에 풀타임으로 참여할 수 있어야 교육 수강이 가능합니다. 


질문 :  다른 교육과 다른 점은?
답변 :  KT 에이블스쿨에서는 이론/실습 교육에 나아가 기업 실전형 프로젝트를 중심으로 실무에서 일하는 방식을 배울 수 있습니다. 또한, KT 현직 전문가가 전담하여 산업지식, AI/DX 기술, 창의성을 갖출 수 있도록 강의와 코칭을 제공하고, 강의 중 궁금한 사항은 전담 튜터가 1:10로 직접 코칭을 제공합니다. 특히, 우수 수료생을 대상으로 KT와 KT 그룹, AI원팀, 협력사 등 채용까지 연계해 드립니다. 


질문 :  출석 조건이 궁금해요.
답변 :  단위기간(훈련시작일로부터 1개월) 내 80% 이상 출석해야 훈련수강 유지가 가능합니다. 
예를 들어, 훈련시작일로부터 1개월을 기준으로 1개월간의 훈련일수가 20일이라고 가정할 경우 16일 이상 출석하셔야 훈련수강을 계속 유지할 수 있습니다. 
결석 기준은 1. 당일 소정훈련시간의 50퍼센트 미만을 수강한 경우, 2. 지각, 조퇴 3회 누적 시, 결석 처리 됩니다. 
훈련 수준 유지를 위해 100% 출석을 권고 드립니다. 

K-Digital Training (K-DT) 규정상 월 1회 휴가 사용이 가능합니다. 


질문 :  나 오늘 상 탔어요!
답변 :  기분 좋겠어요. 


질문 :  아니 기분이 별로야..
답변 :  저랑 놀아요. 


질문 :  이제 공부 좀 해야겠어.
답변 :  공부는 뭐든 좋아요. 


종료합니다.


#### 3) test 데이터에 대해서 성능 측정하기

test 데이터 전체에 대해서 성능을 측정해 봅시다.

In [ ]:
test_data_vecs = get_dataset(test_sent, wv_model, 300)

test_x = test_data_vecs
test_y = test['intent']

In [ ]:
test_y_pred_lgbm=model_lgbm.predict(test_x)
print(confusion_matrix(test_y, test_y_pred_lgbm))
print(classification_report(test_y, test_y_pred_lgbm))

[[0 0 0 ... 0 0 2]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00         5
           2       0.00      0.00      0.00         5
           3       0.00      0.00      0.00        12
           4       0.00      0.00      0.00         3
           5       0.00      0.00      0.00         3
           6       0.00      0.00      0.00         3
           7       0.00      0.00      0.00         3
           8       0.04      0.19      0.06        21
           9       0.00      0.00      0.00         3
          10       0.00      0.00      0.00         3
          11       0.00      0.00      0.00         3
          12       0.00      0.00      0.00         3
          13       0.00      0.00      0.00         7
          14       0.00      0.00      0.00         3
          15       0.00      0.00      0.00         3
          16   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
test_y_pred_rf=model_rf.predict(test_x)
print(confusion_matrix(test_y, test_y_pred_rf))
print(classification_report(test_y, test_y_pred_rf))

[[5 0 0 ... 0 0 0]
 [0 3 0 ... 0 0 0]
 [0 0 9 ... 0 0 0]
 ...
 [0 0 0 ... 5 0 0]
 [0 0 0 ... 0 6 0]
 [0 0 0 ... 0 0 6]]
              precision    recall  f1-score   support

           1       0.71      1.00      0.83         5
           2       0.43      0.60      0.50         5
           3       0.69      0.75      0.72        12
           4       0.33      0.67      0.44         3
           5       0.60      1.00      0.75         3
           6       1.00      0.67      0.80         3
           7       0.50      0.33      0.40         3
           8       0.43      0.43      0.43        21
           9       0.00      0.00      0.00         3
          10       1.00      1.00      1.00         3
          11       1.00      1.00      1.00         3
          12       1.00      0.67      0.80         3
          13       0.33      0.14      0.20         7
          14       0.50      0.33      0.40         3
          15       1.00      0.33      0.50         3
          16   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 추가 모델
* **Word2Vec + 딥러닝**
* **FastText + 딥러닝 / RF**
    * 원본 데이터의 다양성이 부족하다. 이를 보완한 증강 결과 또한 비적합한 정보가 포함되어 있어 성능의 한계가 있는 것으로 보인다.  
    워드 임베딩 모델의 차이는 성능 면에서 크게 두드러지지 않는다.  
    그러나, 직접 설계한 딥러닝 모델보다 랜덤 포레스트 모델 활용 시 더 좋은 성능을 보여준다.  

#### 1) Word2Vec + 딥러닝

In [23]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

y_train = to_categorical(y_train - 1, num_classes=53)
y_val = to_categorical(y_val - 1, num_classes=53)

model_dl = Sequential([
    Dense(128, input_dim=300, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(53, activation='softmax')
])

model_dl.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_dl.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_val, y_val))

Epoch 1/20
379/379 [==============================] - 3s 6ms/step - loss: 2.7144 - accuracy: 0.3081 - val_loss: 1.5472 - val_accuracy: 0.6143
Epoch 2/20
379/379 [==============================] - 1s 3ms/step - loss: 1.6983 - accuracy: 0.5321 - val_loss: 1.0393 - val_accuracy: 0.7300
Epoch 3/20
379/379 [==============================] - 1s 3ms/step - loss: 1.3619 - accuracy: 0.6202 - val_loss: 0.8108 - val_accuracy: 0.8001
Epoch 4/20
379/379 [==============================] - 1s 3ms/step - loss: 1.2069 - accuracy: 0.6581 - val_loss: 0.7167 - val_accuracy: 0.8212
Epoch 5/20
379/379 [==============================] - 1s 3ms/step - loss: 1.0756 - accuracy: 0.6885 - val_loss: 0.6317 - val_accuracy: 0.8367
Epoch 6/20
379/379 [==============================] - 1s 3ms/step - loss: 1.0079 - accuracy: 0.7109 - val_loss: 0.5879 - val_accuracy: 0.8427
Epoch 7/20
379/379 [==============================] - 1s 4ms/step - loss: 0.9776 - accuracy: 0.7167 - val_loss: 0.5489 - val_accuracy: 0.8519
Epoch 

In [36]:
y_test = to_categorical(test['intent'] - 1, num_classes=53)

loss, accuracy = model_dl.evaluate(input_vecs, y_test)
print(f'test Loss: {loss}')
print(f'test Accuracy: {accuracy}')

13/13 [==============================] - 0s 3ms/step - loss: 1.2807 - accuracy: 0.7000
test Loss: 1.280715823173523
test Accuracy: 0.699999988079071


#### 2) FastText + 딥러닝

In [39]:
from gensim.models import FastText
from sklearn.model_selection import train_test_split

ft_model = FastText(sentences=train_sent, vector_size=300, window=5, min_count=3, sg=1)

def get_fasttext_embeddings(sentences, model, vector_size=300):
    embeddings = np.zeros((len(sentences), vector_size))
    for i, sentence in enumerate(sentences):
        word_vectors = [model.wv[word] for word in sentence if word in model.wv]
        if word_vectors:
            embeddings[i] = np.mean(word_vectors, axis=0)
        else:
            embeddings[i] = np.zeros(vector_size)
    return embeddings

train_data_vecs = get_fasttext_embeddings(train_sent, ft_model, 300)

x = train_data_vecs
y = train['intent']

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, shuffle=True, stratify=train['intent'], random_state=42)

y_train = to_categorical(y_train - 1, num_classes=53)
y_val = to_categorical(y_val - 1, num_classes=53)

model_ft = Sequential([
    Dense(128, input_dim=300, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(53, activation='softmax')
])

model_ft.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model_ft.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_val, y_val))

Epoch 1/20
379/379 [==============================] - 3s 6ms/step - loss: 2.7021 - accuracy: 0.3109 - val_loss: 1.5875 - val_accuracy: 0.6319
Epoch 2/20
379/379 [==============================] - 2s 5ms/step - loss: 1.6852 - accuracy: 0.5370 - val_loss: 1.0262 - val_accuracy: 0.7505
Epoch 3/20
379/379 [==============================] - 2s 5ms/step - loss: 1.3485 - accuracy: 0.6229 - val_loss: 0.8166 - val_accuracy: 0.8057
Epoch 4/20
379/379 [==============================] - 1s 3ms/step - loss: 1.1646 - accuracy: 0.6707 - val_loss: 0.6984 - val_accuracy: 0.8169
Epoch 5/20
379/379 [==============================] - 1s 3ms/step - loss: 1.0522 - accuracy: 0.6980 - val_loss: 0.6244 - val_accuracy: 0.8358
Epoch 6/20
379/379 [==============================] - 1s 3ms/step - loss: 0.9817 - accuracy: 0.7139 - val_loss: 0.5782 - val_accuracy: 0.8477
Epoch 7/20
379/379 [==============================] - 1s 4ms/step - loss: 0.9370 - accuracy: 0.7258 - val_loss: 0.5460 - val_accuracy: 0.8513
Epoch 

In [41]:
test_data_vecs = get_fasttext_embeddings(test_sent, ft_model, 300)
y_test = to_categorical(test['intent'] - 1, num_classes=53)

loss, accuracy = model_ft.evaluate(test_data_vecs, y_test)
print(f'test Loss: {loss}')
print(f'test Accuracy: {accuracy}')

13/13 [==============================] - 0s 3ms/step - loss: 1.2305 - accuracy: 0.6769
test Loss: 1.2305197715759277
test Accuracy: 0.6769230961799622


#### 3) FastText + RF

In [43]:
from gensim.models import FastText
from sklearn.model_selection import train_test_split

ft_model = FastText(sentences=train_sent, vector_size=300, window=5, min_count=3, sg=1)

def get_fasttext_embeddings(sentences, model, vector_size=300):
    embeddings = np.zeros((len(sentences), vector_size))
    for i, sentence in enumerate(sentences):
        word_vectors = [model.wv[word] for word in sentence if word in model.wv]
        if word_vectors:
            embeddings[i] = np.mean(word_vectors, axis=0)
        else:
            embeddings[i] = np.zeros(vector_size)
    return embeddings

train_data_vecs = get_fasttext_embeddings(train_sent, ft_model, 300)

x = train_data_vecs
y = train['intent']

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, shuffle=True, stratify=train['intent'], random_state=42)

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

model_rf = RandomForestClassifier(n_estimators=100, random_state=42)

model_rf.fit(x_train, y_train)

y_pred = model_rf.predict(x_val)
accuracy = accuracy_score(y_val, y_pred)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.9785194976867151


In [44]:
test_data_vecs = get_fasttext_embeddings(test_sent, ft_model, 300)
y_test = test['intent']

accuracy = model_rf.score(test_data_vecs, y_test)
print(f'test Accuracy: {accuracy}')

test Accuracy: 0.7282051282051282


* **
* 최종 정확도 : 73%